In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [2]:
import sys

linewidth = 100
formatter = {
    'float': lambda num: f'{num:5.1f}',
    'int': lambda num: f'{float(num):5.1f}',
}
print_args = {
#     'threshold': sys.maxsize,
    'threshold': 100,
    'linewidth': 100,
    'formatter': formatter,
    'edgeitems': 6,
}


In [3]:
import numpy as np
from qtmpy.constants import RYDBERG

alat = 5.107
latvec_alat = 0.5 * np.array([
    [ 1,  1,  1],
    [-1,  1,  1],
    [-1, -1,  1]
])

ecutwfn = 40 * RYDBERG

In [4]:
from qtmpy.lattice import RealLattice, ReciLattice

reallat = RealLattice.from_alat(alat, *latvec_alat)
recilat = ReciLattice.from_reallat(reallat)

print(reallat.axes_alat)
print(recilat.tpiba)
print(recilat.axes_tpiba)

([0.5, 0.5, 0.5], [-0.5, 0.5, 0.5], [-0.5, -0.5, 0.5])
1.2303084603837058
([1.0, 1.1102230246251565e-16, 1.0], [-1.0000000000000002, 1.0000000000000002, 8.695696296261264e-17], [0.0, -1.0, 1.0])


In [5]:
from qtmpy.gspace.gspc import GSpace

gspc_rho = GSpace(recilat, 4 * ecutwfn)
print(gspc_rho.grid_shape)
print(gspc_rho.size)
with np.printoptions(**print_args):
    for i in range(3):
        print(f'G_{i+1}: ', gspc_rho.g_cryst[i])
    print(f'G^2: ', gspc_rho.g_norm2)

(18, 18, 18)
2243
G_1:  [  0.0   0.0   0.0   0.0   0.0   0.0 ...  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0]
G_2:  [  0.0   0.0   0.0   0.0   0.0   0.0 ...  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0]
G_3:  [  0.0   1.0   2.0   3.0   4.0   5.0 ...  -6.0  -5.0  -4.0  -3.0  -2.0  -1.0]
G^2:  [  0.0   3.0  12.1  27.2  48.4  75.7 ... 112.0  78.7  51.5  30.3  15.1   6.1]


In [6]:
from qtmpy.field.field_mixin import QTMBuffer
    
gspc = gspc_rho

arr = QTMBuffer(gspc, np.empty(gspc.size, dtype='c16'))
print(type(arr), hasattr(arr, 'gspc'))
print('-'*10)

# sl = arr[:5]
# print(type(sl), hasattr(sl, 'gspc'))
# val = np.linalg.norm(arr)
# print(type(val), hasattr(val, 'gspc'))

<class 'qtmpy.field.field_mixin.QTMBuffer'> True
----------


In [31]:
a1 = np.empty(gspc.size, dtype='c16')
a2 = np.empty((5, gspc.size), dtype='c16')
print(np.sum(a1 + a2))

x = QTMBuffer(gspc, a1)
y = QTMBuffer(gspc, a2)
print(type(a1), type(a2))
b = x + y
c = np.sum(b, axis=0)
print(c)
print(c.array - np.sum(a1 + a2, axis=0))

(nan+nanj)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[ 0.+0.j  0.+0.j nan+0.j ...  0.+0.j  0.+0.j  0.+0.j]


In [131]:
from qtmpy.gspace import GSpaceBase
import numpy as np

class Buffer(np.ndarray):
    
    ndarray = np.ndarray
    
    @staticmethod
    def _get_basis(arr: np.ndarray, gspc: GSpaceBase):
        shape = getattr(arr, 'shape', None)
        if shape is None:
            return
        
        if arr.shape[-1] == gspc.size:
            basis_type = 'g'
            basis_shape = (gspc.size, )
        elif arr.shape[-3:] == gspc.grid_shape:
            basis_type = 'r'
            basis_shape = gspc.grid_shape
        else:
            return
        
        basis_ndim = len(basis_shape)
        
        dtype = getattr(arr, 'dtype', None)
        if dtype is None:
            return
        
        basis_strides = []
        stride = dtype.itemsize
        for dim in reversed(basis_shape):
            basis_strides.append(stride)
            stride *= dim
        basis_strides = tuple(basis_strides)
        if arr.strides[-basis_ndim:] != basis_strides:
            return
        
        return basis_type, basis_shape, basis_ndim, basis_strides
            
    def __new__(cls, input_array, gspc):
        print("New")
        dtype = getattr(input_array, 'dtype', None)
        if dtype is None:
            raise ValueError("array does not have attribute 'dtype'.")
        if dtype != 'c16':
            raise TypeError("dtype of array must be 'complex128'. "
                            f"got dtype = {dtype}")
            
        if not isinstance(gspc, GSpace):
            raise TypeError("'gspc' must be a 'GSpaceBase' instance. "
                            f"got type {type(gspc)}")
        basis = cls._get_basis(input_array, gspc)
        
        if basis is None:
            return input_array
        obj = np.asarray(input_array, dtype='c16').view(cls)
        obj.gspc = gspc
        (obj._basis_type, obj._basis_shape, 
         obj._basis_ndim, obj._basis_strides) = basis
        return obj
    
    def __array_finalize__(self, obj):
        print("Finalize: ", type(obj))
        if obj is None:
            return
        self.gspc = getattr(obj, 'gspc', None)
        self._basis_type = getattr(obj, '_basis_type', None)
        self._basis_shape = getattr(obj, '_basis_shape', None)
        self._basis_ndim = getattr(obj, '_basis_ndim', None)
        self._basis_strides = getattr(obj, '_basis_strides', None)
        
    @property
    def basis_type(self):
        return self._basis_type
    
    @property
    def basis_shape(self):
        return self._basis_shape
    
    @property
    def basis_ndim(self):
        return self._basis_ndim
    
    @property
    def basis_strides(self):
        return self._basis_strides
    
    def __array_ufunc__(self, ufunc, method, *inputs, out=None, **kwargs):
        args = []
        in_no = []
        for i, input_ in enumerate(inputs):
            if isinstance(input_, A):
                in_no.append(i)
                args.append(input_.view(np.ndarray))
            else:
                args.append(input_)

        outputs = out
        out_no = []
        if outputs:
            out_args = []
            for j, output in enumerate(outputs):
                if isinstance(output, Buffer):
                    out_no.append(j)
                    out_args.append(output.view(np.ndarray))
                else:
                    out_args.append(output)
            kwargs['out'] = tuple(out_args)
        else:
            outputs = (None,) * ufunc.nout

        out = super().__array_ufunc__(ufunc, method, *args, **kwargs)
        if out is NotImplemented:
            return NotImplemented

        print(out)
        if out is not None:
            print('ufunc basis: ', basis)
            if basis is None:
                return np.asarray(out)
            return Buffer(out, self.gspc)
        
    def __getitem__(self, item):
        arr_sl = np.asarray(self).__getitem__(item)
        print('get', type(self), type(arr_sl))
        basis = self._get_basis(arr_sl, self.gspc)
        if basis is None:
            return np.asarray(self).__getitem__(item)
        return super().__getitem__(item)

In [132]:
# arr = Buffer(np.zeros(10))
# print(type(arr), hasattr(arr, 'gspc'))
# print('-'*10)
gspc = gspc_rho

arr = Buffer(np.empty(gspc.size, dtype='c16'), gspc)
print(type(arr), hasattr(arr, 'gspc'))
print('-'*10)

sl = arr[:5]
print(type(sl), hasattr(sl, 'gspc'))
val = np.linalg.norm(arr)
print(type(val), hasattr(val, 'gspc'))

New
Finalize:  <class 'numpy.ndarray'>
<class '__main__.Buffer'> True
----------
get <class '__main__.Buffer'> <class 'numpy.ndarray'>
<class 'numpy.ndarray'> False
<class 'numpy.float64'> False


In [133]:
print(type(np.asarray(arr)))
arr.base is np.asarray(arr).base

<class 'numpy.ndarray'>


True

In [134]:
a1 = Buffer(np.empty(gspc.size, dtype='c16'), gspc)
a2 = Buffer(np.empty((5, gspc.size), dtype='c16'), gspc)
print(type(a1), type(a2))
b = a1 + a2
print('sum', type(b), hasattr(b, 'gspc'))
c = b[0]
print('b[0]', type(c), hasattr(c, 'gspc'))
c = b.reshape(-1).copy()
print('reshape', type(c), hasattr(c, 'gspc'))

New
Finalize:  <class 'numpy.ndarray'>
New
Finalize:  <class 'numpy.ndarray'>
<class '__main__.Buffer'> <class '__main__.Buffer'>


NameError: name 'A' is not defined

In [16]:
import numpy as np
a = np.arange(24).reshape(2,3,4)
print(a.strides)

basis_strides = []
stride = a.dtype.itemsize
for dim in reversed(a.shape):
    print(stride)
    basis_strides.append(stride)
    stride *= dim
basis_strides = tuple(reversed(basis_strides))
print(basis_strides)

(96, 32, 8)
8
32
96
(96, 32, 8)
